# Handle with NaN values

In [124]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    plot_confusion_matrix,
    classification_report)
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
df = pd.read_csv("hotel_bookings.csv")

df['type_of_meal_plan'] = df['type_of_meal_plan'].map({
        'Not Selected': 0, 'Meal Plan 1': 1, 'Meal Plan 2': 2, 'Meal Plan 3': 3
    })
df['market_segment_type'] = df['market_segment_type'].map({
        "Offline": 0,"Online": 1,  "Corporate": 2,  "Aviation": 3 ,"Complementary": 4
    })
df['booking_status'] = df['booking_status'].map({
        "Not_Canceled": 0,"Canceled": 1
    })
df['room_type_reserved'] = df['room_type_reserved'].map({
        "Room_Type 1": 0,"Room_Type 2": 1,  "Room_Type 3": 2,  "Room_Type 4": 3 ,"Room_Type 5": 4
    ,"Room_Type 6": 5,"Room_Type 7": 6
    })

df.drop(columns=["Booking_ID"],inplace=True)

In [125]:

X = df.drop(['booking_status'],axis=1)
Y = df['booking_status']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, shuffle=True,random_state=42)

In [248]:
def impute_function(X_train,X_test,y_train,y_test,imputer,col):

    train = X_train.copy()
    test = X_test.copy()
    
    train.iloc[:,:] = imputer.fit_transform(train)
    test.iloc[:,:] = imputer.transform(test)

    rf = RandomForestClassifier(random_state=42).fit(train, y_train)
    prediction=rf.predict(test)

    acc = accuracy_score(y_test,prediction)
    f1 = f1_score(y_test,prediction)
    prec = precision_score(y_test,prediction)
    rec = recall_score(y_test,prediction)
    

    results = pd.DataFrame([[col,'Random Forrest', acc, f1, prec, rec]],
                         columns =  ["impute",'Model','Accuracy Score','F1 score','Precision','Recall'])
    return results

In [249]:

df_result_SimpleImputer=pd.DataFrame(columns =  ["impute",'Model','Accuracy Score','F1 score','Precision','Recall'])
for col in ["mean","median","most_frequent","constant"]:
    results = impute_function(X_train,X_test,y_train,y_test,SimpleImputer(strategy=col),col)
    df_result_SimpleImputer= pd.concat([df_result_SimpleImputer,results],axis=0)
    
df_result_KNNImputer=pd.DataFrame(columns =  ["impute",'Model','Accuracy Score','F1 score','Precision','Recall'])
for col in [2]:
    results = impute_function(X_train,X_test,y_train,y_test,KNNImputer(n_neighbors=col, weights="uniform"),"knn_neighbor"+str(col))
    df_result_KNNImputer= pd.concat([df_result_KNNImputer,results],axis=0)    
    
    
col=  "IterativeImputer"  
df_result_IterativeImputer=pd.DataFrame(columns =  ["impute",'Model','Accuracy Score','F1 score','Precision','Recall'])
results = impute_function(X_train,X_test,y_train,y_test,IterativeImputer(),col)
df_result_IterativeImputer= pd.concat([df_result_IterativeImputer,results],axis=0)    

In [250]:
df_full = pd.concat([df_result_SimpleImputer,df_result_KNNImputer,df_result_IterativeImputer])
df_full.sort_values(by="Accuracy Score",inplace=True,ascending=False)

In [261]:
X_train

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
29580,2.0,0.0,1.0,3.0,1.0,0.0,0.0,200.0,2018.0,8.0,29.0,1.0,0.0,0.0,0.0,90.95,0.0
6325,2.0,0.0,1.0,3.0,1.0,0.0,0.0,79.0,2018.0,3.0,24.0,1.0,0.0,0.0,0.0,90.95,1.0
33899,2.0,0.0,1.0,4.0,1.0,0.0,3.0,78.0,2018.0,4.0,6.0,1.0,0.0,0.0,0.0,99.45,1.0
720,2.0,0.0,2.0,NaN,1.0,0.0,0.0,61.0,2017.0,NaN,4.0,NaN,0.0,0.0,0.0,91.00,0.0
22120,2.0,1.0,0.0,4.0,1.0,NaN,0.0,201.0,2018.0,11.0,1.0,1.0,0.0,0.0,0.0,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16850,2.0,0.0,2.0,2.0,1.0,0.0,0.0,43.0,2017.0,12.0,26.0,0.0,0.0,0.0,0.0,55.00,2.0
6265,1.0,0.0,0.0,2.0,2.0,0.0,0.0,102.0,2017.0,10.0,16.0,1.0,0.0,0.0,0.0,80.00,0.0
11284,2.0,0.0,0.0,2.0,1.0,NaN,NaN,5.0,2018.0,5.0,24.0,2.0,NaN,0.0,0.0,110.00,1.0
860,2.0,0.0,0.0,3.0,1.0,0.0,0.0,213.0,2018.0,6.0,7.0,1.0,0.0,0.0,0.0,130.00,0.0


In [251]:
df_full

,impute,Model,Accuracy Score,F1 score,Precision,Recall
0,knn_neighbor2,Random Forrest,0.898098,0.838267,0.884308,0.796784
0,IterativeImputer,Random Forrest,0.897730,0.837114,0.886547,0.792903
0,mean,Random Forrest,0.897271,0.835926,0.888057,0.789576
0,median,Random Forrest,0.894606,0.832823,0.877996,0.792071
0,most_frequent,Random Forrest,0.893228,0.830216,0.877665,0.787635
0,constant,Random Forrest,0.890563,0.825187,0.876794,0.779318


In [ ]:
constant da hangi değerleri koyuyor?
iterative imputer nasıl çalışır?
missing değerleri yeni sütunda oluşturulduğunda neden daha iyi katkı sağlıyor?
nan sayılarını da gruplayarak sayılarını yeni feature olarak yazabiliriz
discord salona giriş çıkışta sesi kısabiliyor muyuz?


Note: Some algorithms like XGBoost and LightGBM can handle missing values without any preprocessing, by supplying relevant parameters.

https://www.kaggle.com/code/parulpandey/a-guide-to-handling-missing-values-in-python

In [ ]:
[   a   b    c 
    20 false 50
    30 false 60 
    nan true nan true
    40 false 80
]

In [252]:
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_test_plus = X_test.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_test_plus[col + '_was_missing'] = X_test_plus[col].isnull()
    
    
    
knn_imputer = KNNImputer(n_neighbors=2, weights="uniform")
X_train_plus= knn_imputer.fit_transform(X_train_plus)
X_test_plus = knn_imputer.transform(X_test_plus)

rf = RandomForestClassifier(random_state=42).fit(X_train_plus, y_train)
prediction=rf.predict(X_test_plus)

acc = accuracy_score(y_test,prediction)
f1 = f1_score(y_test,prediction)
prec = precision_score(y_test,prediction)
rec = recall_score(y_test,prediction)

results_knn2_plus = pd.DataFrame([["knn_2_plus",'Random Forrest', acc, f1, prec, rec]],
                     columns =  ["impute",'Model','Accuracy Score','F1 score','Precision','Recall'])

https://www.kaggle.com/code/alexisbcook/missing-values

In [259]:
results_knn2_plus

,impute,Model,Accuracy Score,F1 score,Precision,Recall
0,knn_2_plus,Random Forrest,0.898741,0.838132,0.891284,0.790962


In [257]:
df_full_full = pd.concat([df_full,results_knn2_plus])
df_full_full.sort_values(by="Accuracy Score",inplace=True,ascending=False)

In [258]:
df_full_full

,impute,Model,Accuracy Score,F1 score,Precision,Recall
0,knn_2_plus,Random Forrest,0.898741,0.838132,0.891284,0.790962
0,knn_neighbor2,Random Forrest,0.898098,0.838267,0.884308,0.796784
0,IterativeImputer,Random Forrest,0.897730,0.837114,0.886547,0.792903
0,mean,Random Forrest,0.897271,0.835926,0.888057,0.789576
0,median,Random Forrest,0.894606,0.832823,0.877996,0.792071
0,most_frequent,Random Forrest,0.893228,0.830216,0.877665,0.787635
0,constant,Random Forrest,0.890563,0.825187,0.876794,0.779318
